In [1]:
import pandas as pd
import numpy as np
import mysql.connector
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
connection = mysql.connector.connect(
    host = os.getenv('DB_HOST'),
    user = os.getenv('DB_USER'),
    password = os.getenv('DB_PASSWORD'),
    database = os.getenv('DB_DATABASE')
)

In [4]:
cursor = connection.cursor()

In [10]:
df1 = pd.read_csv("./baseFicticia.csv", sep=";")

In [11]:
df1['data_bloqueio'] = pd.to_datetime(df1['data_bloqueio'], format='%d/%m/%Y %H:%M')
df1['data_arquivo'] = pd.to_datetime(df1['data_arquivo'], format='%d/%m/%Y %H:%M')
df1['data_inicio_bloqueio'] = pd.to_datetime(df1['data_inicio_bloqueio'], format='%d/%m/%Y %H:%M')

In [12]:
df1['data_bloqueio'] = df1['data_bloqueio'].dt.date
df1['data_arquivo'] = df1['data_arquivo'].dt.date
df1['data_inicio_bloqueio'] = df1['data_inicio_bloqueio'].dt.date

In [13]:
df1['telefone'] = df1['telefone'].astype(object)  

In [14]:
df1

,origem,telefone,data_bloqueio,data_arquivo,empresa,data_inicio_bloqueio
0,PROCON AL,54123456789,2021-04-09,2023-02-01,TODAS,2021-05-09
1,PROCON AL,65123456789,2017-03-03,2023-02-01,TODAS,2017-04-02
2,PROCON AL,77123456789,2012-10-05,2023-02-01,TODAS,2012-11-04
3,PROCON AL,88123456789,2019-09-02,2023-02-01,TODAS,2019-10-02
4,PROCON AL,99123456789,2010-09-16,2023-02-01,TODAS,2010-10-16
...,...,...,...,...,...,...
1000202,PROCON PR,41997148794,2021-07-08,2023-02-01,TODAS,2021-08-07
1000203,PROCON PR,41997148798,2021-04-20,2023-02-01,TODAS,2021-05-20
1000204,PROCON PR,41997148873,2021-04-06,2023-02-01,TODAS,2021-05-06
1000205,PROCON PR,41997148874,2021-01-27,2023-02-01,TODAS,2021-02-26


In [15]:
df = pd.DataFrame(df1)

In [16]:
df['data_processamento'] = '2023-06-19'

In [17]:
df.columns = df.columns.str.upper()

In [18]:
df.dtypes

ORIGEM                  object
TELEFONE                object
DATA_BLOQUEIO           object
DATA_ARQUIVO            object
EMPRESA                 object
DATA_INICIO_BLOQUEIO    object
DATA_PROCESSAMENTO      object
dtype: object

In [19]:
df

,ORIGEM,TELEFONE,DATA_BLOQUEIO,DATA_ARQUIVO,EMPRESA,DATA_INICIO_BLOQUEIO,DATA_PROCESSAMENTO
0,PROCON AL,54123456789,2021-04-09,2023-02-01,TODAS,2021-05-09,2023-06-19
1,PROCON AL,65123456789,2017-03-03,2023-02-01,TODAS,2017-04-02,2023-06-19
2,PROCON AL,77123456789,2012-10-05,2023-02-01,TODAS,2012-11-04,2023-06-19
3,PROCON AL,88123456789,2019-09-02,2023-02-01,TODAS,2019-10-02,2023-06-19
4,PROCON AL,99123456789,2010-09-16,2023-02-01,TODAS,2010-10-16,2023-06-19
...,...,...,...,...,...,...,...
1000202,PROCON PR,41997148794,2021-07-08,2023-02-01,TODAS,2021-08-07,2023-06-19
1000203,PROCON PR,41997148798,2021-04-20,2023-02-01,TODAS,2021-05-20,2023-06-19
1000204,PROCON PR,41997148873,2021-04-06,2023-02-01,TODAS,2021-05-06,2023-06-19
1000205,PROCON PR,41997148874,2021-01-27,2023-02-01,TODAS,2021-02-26,2023-06-19


In [150]:
chunksize = 300000

# Particione o DataFrame em chunks de 300 registros
chunks = [df[i:i+chunksize] for i in range(0, len(df), chunksize)]

In [ ]:
# Itere sobre os chunks do DataFrame
for chunk in chunks:
    for indice, row in chunk.iterrows():
        # Obtenha os valores de cada coluna para a linha atual
        origem = row['ORIGEM']
        telefone = row['TELEFONE']
        data_bloqueio = row['DATA_BLOQUEIO']
        data_arquivo = row['DATA_ARQUIVO']
        empresa = row['EMPRESA']
        OrigemPedido = row['DATA_INICIO_BLOQUEIO']
        data_inicio_bloqueio = row['DATA_PROCESSAMENTO']

        # Execute a consulta SQL para inserir os dados na tabela
        consulta = "INSERT IGNORE INTO tabela_de_clientes (origem, telefone, data_bloqueio, data_arquivo, empresa, OrigemPedido, data_inicio_bloqueio) VALUES (%s, %s, %s, %s, %s, %s, %s)"
        valores = (origem, telefone, data_bloqueio, data_arquivo, empresa, OrigemPedido, data_inicio_bloqueio)
        cursor.execute(consulta, valores)


In [ ]:
# Confirme as alterações e feche a conexão
conexao.commit()
conexao.close()